In [244]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [245]:
%reload_ext autoreload

In [246]:
import json

with open('grouping_test_eng_19.txt', 'rb') as f:
    request = json.load(f)

In [247]:
from main import handler

res = handler(request, None)

0.2976888020833333


In [231]:
group = json.loads(res['body'])

In [224]:
group

{'group': {'28': [{'id': 'f100dc1313c9423c813529c576827e50',
    'status': 'completed',
    'deleted': False,
    'endTime': '2019-07-04T12:36:44Z',
    'duration': 8,
    'spokenBy': 'd78b6120-3951-4b66-8e5d-5a8f30b9b2a9',
    'createdAt': '2019-07-04T12:36:44.875927844Z',
    'deletedAt': None,
    'startTime': '2019-07-04T12:36:36Z',
    'updatedAt': '2019-07-04T12:36:48.334543504Z',
    'confidence': 0.94158244,
    'recordingId': '34b40afc81b2415ebe52c76deed09947',
    'transcriber': 'deepgram',
    'languageCode': 'en-US',
    'originalText': 'Not this case screen and right now America.',
    'transcriptId': '2ab32670-d684-4a52-bf31-a6c2f29b64e6'}],
  '44': [{'id': '10e951d2895743e8a842e1a9ac73291f',
    'status': 'completed',
    'deleted': False,
    'endTime': '2019-07-04T12:25:33Z',
    'duration': 104,
    'spokenBy': 'd78b6120-3951-4b66-8e5d-5a8f30b9b2a9',
    'createdAt': '2019-07-04T12:25:34.663985674Z',
    'deletedAt': None,
    'startTime': '2019-07-04T12:23:49Z',
    

In [226]:
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-07-04T12:15:14Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  28


Discussion:

 
Minutes from the start of the meeting:  0:21:22 f100dc1313c9423c813529c576827e50 

Not this case screen and right now America. 




PIMs  44


Discussion:

 
Minutes from the start of the meeting:  0:08:35 10e951d2895743e8a842e1a9ac73291f 

And you do that by referring to social norms that's more less you your father from the value of the way, but the way the other are way you do that is how a conversation with yourself about if you don't really know who you are are because you know what you're like you won't do you're told you won't do what you tell yourself to do you must have noticed that it's like you're a bad employee and a worse loss and going into those for you you don't know what you want to do and then when you tell yourself what do you don't do it anyway but you know my point that you have to understand that you're not your own servant. So to speak someone that you have to negotiate with and you someone that you want to present the opportunity of 

In [219]:
groups = []
for i in group['group'].keys():
    if len(group['group'][i])==1:
        continue
    else:
        temp = []
        for seg in group['group'][i]:
            temp.append(seg['originalText'])
        groups.append(' '.join(temp))

In [52]:
import pandas as pd
df = pd.read_csv("../data/processed/master_tag+para_cluster_df.csv")
import ast
tag = [] 
for tags in df['tags']:
    for t in ast.literal_eval(tags):
        tag.append(t)
print ("no of tags present:", len(tag))
unique_tag_unr = list(set(tag))
unique_tag = []
for u in unique_tag_unr:
    if tag.count(u) > 10:
        unique_tag.append(u)
print ("no of unique tags present:", len(unique_tag))

no of tags present: 15918
no of unique tags present: 259


In [128]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_embeddings(input_list, req_data=None):

    if req_data is None:
        lambda_payload = {"body": {"text_input": input_list}}
    else:
        lambda_payload = {"body": {"request": req_data, "text_input": input_list}}

    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="keyphrase_ranker",
            InvocationType="RequestResponse",
            Payload=json.dumps(lambda_payload),
        )

        lambda_output = (
            invoke_response["Payload"].read().decode("utf8").replace("'", '"')
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            embedding_vector = np.asarray(json.loads(response_body)["embeddings"])

        else:
            embedding_vector = np.asarray(json.loads(response_body)["embeddings"])
    except Exception as e:
        print (e)
        pass
    return embedding_vector

In [54]:
fv = get_embeddings(groups, req_data=None)
fv_tag = get_embeddings([i for i in unique_tag if i!=""], req_data=None)

In [55]:
from scipy.spatial.distance import cosine
for index, tex in enumerate(groups):
    closest = -1
    closest_tag = None
    closest_2= -1
    closest_tag_2 = None
    closest_3 = -1
    closest_tag_3 = None
    closest_4= -1
    closest_tag_4 = None
    for index2, t in enumerate([i for i in unique_tag if i!=""]):
        if t!="":
            score = 1 - cosine(fv[index],fv_tag[index2])
            if score > closest:
                closest = score
                closest_tag = t
            elif score > closest_2:
                closest_tag_2 = t
                closest_2 = score
            elif score > closest_3:
                closest_tag_3 = t
                closest_3 = score
            elif score > closest_4:
                closest_tag_4 = t
                closest_4 = score
    print ("\n\n\nsentence: \n\n", tex)
    print ("\n most similar tags: ", str(closest_tag) + " , " + str(closest_tag_2) + " , " + str(closest_tag_3) + " , " + str(closest_tag_4) )




sentence: 

 Ux Designers have a a lot on their plate. They research that wire user tests the list goes on and off if you're new to the game this can all feel very overwhelming well throw that here out the window because we're about to embark on a journey through the sixteenth end of the UX design process that before starting any successful uX project you'll need to be in tune with two things your user in your brain. I start with the user big question around this is what problems are they making once you can identify the problem, then you can begin to start working to towards the solution second is understanding the brain and how does the project you're are working on aligned with the branch mission and goal to summarize in this stage you wanna identify the problems facing the users as well as the core goals of the brand and see how they along as a designer you probably have a gun of a*sumptions about what the user wants or doesn't want but not challenging those a*sumptions channel.